In [ ]:
# append all the json files in one dataframe and remove duplicates
import os

folder = os.listdir('scraped_tweets')

for file in folder:
    df = pd.read_json('scraped_tweets/'+file) #load the file
    print(file, len(df))
    
    data = data.append(df, ignore_index=True)

print('number of tweets before duplicates removal', len(data))
data = data.drop_duplicates(subset = ['text']) # remove duplicate lines
print('number of tweets after duplicates removal', len(data))

In [1]:
pip install cleantext

You should consider upgrading via the '/home/aina/tmp/PATH/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import modules
import pandas as pd

### Cleaning

In [3]:
import string
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


emoji_pattern = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)

en_stopwords = stopwords.words('english')

def clean(text):
    text = text.lower() #lowercase
    text = text.replace('<br /><br />', ' ') # deletes br
    text = text.translate(str.maketrans('', '', string.punctuation)) #deletes punctuation
    text = text.translate(str.maketrans('', '', '1234567890')) #deletes numbers
    text = emoji_pattern.sub(r'', text) #deletes emojis
    
    # Tokenize the tweet
    tokens = word_tokenize(text)
    
    # Remove stopwords
    clean_text = [word for word in tokens if word not in en_stopwords]
    
    return ' '.join(clean_text)

In [5]:
# append all the json files in one dataframe and remove duplicates
import os

folder = os.listdir('scraped_tweets')
data = pd.DataFrame(columns = ['username', 'description', 'date', 'location', 'following', 'followers',
       'totaltweets', 'retweet', 'retweetcount', 'text', 'hashtags'])

for file in folder:
    df = pd.read_json('scraped_tweets/'+file) #load the file
    print(file, len(df))
    
    data = data.append(df, ignore_index=True)

print('number of tweets before duplicates removal', len(data))
data = data.drop_duplicates(subset = ['text']) # remove duplicate lines
print('number of tweets after duplicates removal', len(data))

scraped_tweets_wine.json 1000
scraped_tweets_winereview2.json 65
scraped_tweets_winechat.json 0
scraped_tweets_winelover.json 1326
scraped_tweets_badwine.json 0
scraped_tweets_sonomachat.json 3
scraped_tweets_wine2.json 10000
scraped_tweets_winereview.json 67
scraped_tweets_goodwine.json 6
scraped_tweets_worstwine.json 0
scraped_tweets_bestwine.json 28
scraped_tweets_wine3.json 766
scraped_tweets_winewednesday.json 410
scraped_tweets_wine4.json 5171
number of tweets before duplicates removal 18842
number of tweets after duplicates removal 6524


In [9]:
# clean all the tweets in the dataframe
clean_text = []
for text in data['text']:
    
    c_text = clean(text)
    clean_text.append(c_text)
    
data['text'] = clean_text

In [10]:
# save the clean dataframe in a file
data.to_json('clean_tweets.json')

### Load Clean Dataset

In [11]:
df = pd.read_json("clean_tweets.json")
print('dataframe length:', len(df))
df.columns

dataframe length: 6524


Index(['username', 'description', 'date', 'location', 'following', 'followers',
       'totaltweets', 'retweet', 'retweetcount', 'text', 'hashtags'],
      dtype='object')

### Frequency of words

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/aina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer() 
lemmatizer = WordNetLemmatizer()

# Counts all the occurances of words in the given set of sentences
# Returns a dict {word: #occurrences} for stemming and one for lemmatization
# and the total number of occurrences
def counter(s):
    
    d_stem = {} #empty dict
    d_lemma = {}
    total = 0 # number of seen words
    
    for sent in s['text']:
        
        text = word_tokenize(sent) # Tokenize the text
        text = list(filter(lambda a: a != '', text)) # removes empty elements
        
        for w in text:
            total += 1
            
            w_s = stemmer.stem(w)
            w_l = lemmatizer.lemmatize(w)
            
            if w_s in d_stem: 
                d_stem[w_s] += 1
            else:
                d_stem[w_s] = 1
                
            if w_l in d_lemma: 
                d_lemma[w_l] += 1
            else:
                d_lemma[w_l] = 1
                
    return d_stem, d_lemma, total

In [13]:
d_stem, d_lemma, total = counter(df)

In [14]:
print('Total number of differentwords (stemming):', len(d_stem))
print('Total number of differentwords (lemmatization):', len(d_lemma))
print('Num of seen words', total)

sorted_d_stem = sorted(d_stem.items(), key=lambda item: item[1], reverse=True)
sorted_d_lemma = sorted(d_lemma.items(), key=lambda item: item[1], reverse=True)

print(sorted_d_stem[:50])
print()
print(sorted_d_lemma[:50])

Total number of differentwords (stemming): 26322
Total number of differentwords (lemmatization): 28070
Num of seen words 125028
[('wine', 9665), ('winelov', 1184), ('bottl', 852), ('’', 676), ('amp', 670), ('glass', 625), ('art', 616), ('tast', 519), ('winetast', 496), ('etsysel', 478), ('set', 473), ('food', 469), ('beer', 464), ('winemak', 440), ('wineri', 435), ('new', 410), ('vineyard', 407), ('drink', 402), ('gift', 398), ('layer', 384), ('vintag', 383), ('redwin', 362), ('base', 342), ('repurposedupcycl', 338), ('red', 338), ('us', 335), ('travel', 317), ('love', 316), ('winewednesday', 313), ('vino', 298), ('champagn', 298), ('lover', 295), ('great', 291), ('scene', 278), ('day', 269), ('ad', 268), ('grape', 260), ('delici', 253), ('whitewin', 252), ('good', 251), ('whiski', 248), ('today', 247), ('get', 242), ('make', 240), ('one', 237), ('gtgtgt', 230), ('harvest', 226), ('chardonnay', 224), ('pinot', 223), ('enjoy', 221)]

[('wine', 9661), ('bottle', 835), ('winelover', 819),

In [16]:
for ele in sorted_d_stem[:100]:
    print(ele)

('wine', 9665)
('winelov', 1184)
('bottl', 852)
('’', 676)
('amp', 670)
('glass', 625)
('art', 616)
('tast', 519)
('winetast', 496)
('etsysel', 478)
('set', 473)
('food', 469)
('beer', 464)
('winemak', 440)
('wineri', 435)
('new', 410)
('vineyard', 407)
('drink', 402)
('gift', 398)
('layer', 384)
('vintag', 383)
('redwin', 362)
('base', 342)
('repurposedupcycl', 338)
('red', 338)
('us', 335)
('travel', 317)
('love', 316)
('winewednesday', 313)
('vino', 298)
('champagn', 298)
('lover', 295)
('great', 291)
('scene', 278)
('day', 269)
('ad', 268)
('grape', 260)
('delici', 253)
('whitewin', 252)
('good', 251)
('whiski', 248)
('today', 247)
('get', 242)
('make', 240)
('one', 237)
('gtgtgt', 230)
('harvest', 226)
('chardonnay', 224)
('pinot', 223)
('enjoy', 221)
('octob', 214)
('green', 212)
('theriotstudio', 207)
('rate', 207)
('nft', 206)
('oz', 205)
('liquor', 202)
('year', 196)
('via', 195)
('like', 195)
('look', 195)
('time', 191)
('commun', 189)
('star', 187)
('recip', 186)
('best', 18